In [ ]:
!pip install torch -Uq
!pip install transformers -Uq
!pip install tiktoken -Uq
!pip install datasets -Uq
!pip install matplotlib -Uq

In [ ]:
import json
import torch
import torch.nn as nn
import plotly.graph_objects as go
import plotly.offline

In [ ]:
class Tokenizer:
  def __init__(self, vocab_path):
    with open(vocab_path, "r", encoding="utf-8") as f:
      self.vocab = json.load(f)
      self.reverse_vocab = {v: k for k, v in self.vocab.items()}

  def encode(self, text):
    tokens = []

    for word in text.split():
      i = 0
      while i < len(word):
        found_match = False
        for j in range(len(word), i, -1):
          subword = word[i:j]
          if subword in self.vocab:
            tokens.append(self.vocab[subword])
            i = j
            found_match = True
            break
        if not found_match:
          tokens.append(self.vocab.get("<unk>", 0))
          i += 1
      tokens.append(self.vocab.get(" ", 1))
    if tokens:
      tokens.pop()
    return tokens

  def tokenize(self, text):
    token_ids = self.encode(text)
    return [self.reverse_vocab[id] for id in token_ids]

  def decode(self, ids):
    text = ""
    for id in ids:
      text += self.reverse_vocab.get(id, "<unk>")
    return text

In [ ]:
def get_rotary_position_encoding(input: torch.Tensor, base=10000, device="cpu"):
  context_length, dimension = input.shape

  assert dimension % 2 == 0, "boyutlar eşit olmalıdır"

  half_dimension = dimension // 2

  freqs_indices = torch.arange(0, half_dimension, device=device, dtype=torch.float32)
  freqs = 1.0 / (base ** (freqs_indices / dimension))

  positions = torch.arange(0, context_length, device=device, dtype=torch.float32).unsqueeze(1)

  angles = positions * freqs

  sin_angles = torch.sin(angles)
  cos_angles = torch.cos(angles)

  input_even = input[:, :dimension // 2]
  input_odd = input[:, dimension // 2:]

  input_even_rotated = input_even * cos_angles - input_odd * sin_angles
  input_odd_rotated = input_even * sin_angles + input_odd * cos_angles

  input_rotated = torch.empty_like(input)

  input_rotated[:, :dimension // 2] = input_even_rotated
  input_rotated[:, dimension // 2:] = input_odd_rotated

  return input_rotated

In [ ]:
class UstaModel(nn.Module):
  def __init__(self, vocab_size, embedding_dim, context_length):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim)

    self.get_pos = get_rotary_position_encoding

  def forward(self, x):
    x = self.embedding(x)
    x = self.get_pos(x)
    return x

In [ ]:
def plot_dots(sentences_data, title, dims=[0, 1, 2]):
  data = [
    go.Scatter3d(
      x=sentence_data["words"][:, dims[0]],
      y=sentence_data["words"][:, dims[1]],
      z=sentence_data["words"][:, dims[2]],
      mode="markers+text",
      marker=dict(
        size=6,
        color=sentence_data["color"],
      ),
      text=sentence_data["labels"],
      hoverinfo="text",
    ) for sentence_data in sentences_data
  ]

  layout = go.Layout(
    scene=dict(
      xaxis_title="Hardness",
      yaxis_title="Brightness",
      zaxis_title="Redness",
    ),
    title=title,
    width=1000,
    height=1000,
  )

  fig = go.Figure(data=data, layout=layout)
  plotly.offline.iplot(fig)

##Tekrar Edelim

In [ ]:
tokenizer = Tokenizer("tokenizer_1.json")

In [ ]:
prompt = "the capital of united states and the capital of france"

In [ ]:
tokens = tokenizer.encode(prompt)

In [ ]:
x = torch.tensor(tokens, dtype=torch.long)

In [ ]:
torch.manual_seed(42)

In [ ]:
model = UstaModel(vocab_size=len(tokenizer.vocab), embedding_dim=4, context_length=32)

In [ ]:
model(x)

In [ ]:
sentence_meanings = model(x)

In [ ]:
sentence_meanings.shape

In [ ]:
sentences = [
  {
    "words": sentence_meanings.detach().numpy(),
    "labels": tokenizer.tokenize(prompt),
    "color": "red",
  },
]

plot_dots(sentences, "Models Context Space")

# Attention

Her kelimenin diğer kelimelere ne kadar dikkat etmesi gerektiğini” sayısal olarak hesaplar

##Manhattan Mesafe Ölçeği

In [ ]:
sentence_meanings

In [ ]:
the_pos = [1.9269,  1.4873,  0.9007, -2.1055]
capital_pos = [-0.2432, -0.8911,  0.6348, -1.8179]

In [ ]:
hardness_dist = the_pos[0] - capital_pos[0]
brightness_dist = the_pos[1] - capital_pos[1]
redness_dist = the_pos[2] - capital_pos[2]
blue_dist = the_pos[3] - capital_pos[3]

In [ ]:
hardness_dist, brightness_dist, redness_dist, blue_dist

In [ ]:
total_dis = abs(hardness_dist) + abs(brightness_dist) + abs(redness_dist) + abs(blue_dist)

In [ ]:
total_dis

##Cosinus Benzerliği Mesafe Ölçümü

Originden iki kelime arasındaki açının kosinüsünü alıp benzerliğini ölçer

Gerçek Manhatten ya da Cosinus benzerliği kullanmak zorunda değiliz

Önceden belirlediğimiz herhangi bir kurala göre eğitirsek de aynı noktaya varacağız

In [ ]:
the_pos = [1.9269,  1.4873,  0.9007, -2.1055]
capital_pos = [-0.2432, -0.8911,  0.6348, -1.8179]

In [ ]:
cos_sim_hardness = the_pos[0] * capital_pos[0]
cos_sim_brightness = the_pos[1] * capital_pos[1]
cos_sim_redness = the_pos[2] * capital_pos[2]
cos_sim_blue = the_pos[3] * capital_pos[3]

In [ ]:
total_cos_sim = cos_sim_hardness + cos_sim_brightness + cos_sim_redness + cos_sim_blue

In [ ]:
cos_sim_hardness, cos_sim_brightness, cos_sim_redness, cos_sim_blue, total_cos_sim

In [ ]:
sentence_meanings[0], sentence_meanings[1],  sentence_meanings[2],  sentence_meanings[3]

In [ ]:
cs_0_0 = sentence_meanings[0][0] * sentence_meanings[0][0] + sentence_meanings[0][1] * sentence_meanings[0][1] + sentence_meanings[0][2] * sentence_meanings[0][2] + sentence_meanings[0][3] * sentence_meanings[0][3]
cs_0_1 = sentence_meanings[0][0] * sentence_meanings[1][0] + sentence_meanings[0][1] * sentence_meanings[1][1] + sentence_meanings[0][2] * sentence_meanings[1][2] + sentence_meanings[0][3] * sentence_meanings[1][3]
cs_0_2 = sentence_meanings[0][0] * sentence_meanings[2][0] + sentence_meanings[0][1] * sentence_meanings[2][1] + sentence_meanings[0][2] * sentence_meanings[2][2] + sentence_meanings[0][3] * sentence_meanings[2][3]
cs_0_3 = sentence_meanings[0][0] * sentence_meanings[3][0] + sentence_meanings[0][1] * sentence_meanings[3][1] + sentence_meanings[0][2] * sentence_meanings[3][2] + sentence_meanings[0][3] * sentence_meanings[3][3]

In [ ]:
cs_0_0, cs_0_1, cs_0_2, cs_0_3

In [ ]:
the_similarities = []

for i in range(sentence_meanings.shape[0]):
  cs_the_i = sentence_meanings[0][0] * sentence_meanings[i][0] + sentence_meanings[0][1] * sentence_meanings[i][1] + sentence_meanings[0][2] * sentence_meanings[i][2] + sentence_meanings[0][3] * sentence_meanings[i][3]
  the_similarities.append(cs_the_i)

In [ ]:
the_similarities

Torch üzerinde işlem yapmadan önce boş yerlerini oluşturmamız gerekir

In [ ]:
all_similarities = torch.zeros(sentence_meanings.shape[0], sentence_meanings.shape[0])
for j in range(sentence_meanings.shape[0]):
  j_similarities = torch.zeros(sentence_meanings.shape[0])

  for i in range(sentence_meanings.shape[0]):
    for k in range(sentence_meanings.shape[1]):
      cs_j_i = sentence_meanings[j][k] * sentence_meanings[i][k]
      j_similarities[i] += cs_j_i

  all_similarities[j] = j_similarities

In [ ]:
all_similarities.detach().numpy()

In [ ]:
all_similarities.shape

In [ ]:
sentence_meanings.shape, sentence_meanings.T.shape

In [ ]:
all_sim_torch = sentence_meanings @ sentence_meanings.T

In [ ]:
all_sim_torch.shape

In [ ]:
all_sim_torch

Doğasını bozmadan daha yönetilebilir bir hale getirilmesi normalizasyon sayesinde gerçekleşir

In [ ]:
# attention_weights = all_similarities / all_similarities.sum(dim=1, keepdim=True)

In [ ]:
attention_weights = torch.softmax(all_similarities, dim=1)

In [ ]:
attention_weights

In [ ]:
torch.sum(all_sim_torch[0]), torch.sum(attention_weights[0])

In [ ]:
sentence_context_vector = attention_weights @ sentence_meanings

In [ ]:
sentence_context_vector

In [ ]:
sentence_meanings

In [ ]:
sentence_meaning_without_pos = model.embedding(x)

##Simple Self Attention

In [ ]:
sentences = [
  {
    "words": sentence_meanings.detach().numpy(),
    "labels": tokenizer.tokenize(prompt),
    "color": "purple",
  },
  {
    "words": sentence_context_vector.detach().numpy(),
    "labels": tokenizer.tokenize(prompt),
    "color": "orange",
  },
  {
    "words": sentence_meaning_without_pos.detach().numpy(),
    "labels": tokenizer.tokenize(prompt),
    "color": "blue",
  },
]

plot_dots(sentences, "Models Attention Sentence Space")